# Learning by doing: Va a llover o no va a llover? 

Paso a paso el analisis completo para predecir si un dia va a llover o no. 


In [33]:
weather <- read.csv("https://raw.githubusercontent.com/frm1789/Weather_2019/master/weather_2014.csv",sep=";",stringsAsFactors=FALSE)

## Seccion A) Formateo de datos


In [34]:
dim(weather)

[1] 365  14

In [35]:
head(weather)

day.count,day,month,season,l.temp,h.temp,ave.temp,l.temp.time,h.temp.time,rain,ave.wind,gust.wind,gust.wind.time,dir.wind
1,1,1,Winter,12.7,14.0,13.4,01:25,23:50,32.0,11.4,53.1,15:45,S
2,2,1,Winter,11.3,14.7,13.5,07:30,11:15,64.8,5.6,41.8,22:25,S
3,3,1,Winter,12.6,14.7,13.6,21:00,14:00,12.7,4.3,38.6,00:00,SSW
4,4,1,Winter,7.7,13.9,11.3,10:35,01:50,20.1,10.3,66.0,09:05,SW
5,5,1,Winter,8.8,14.6,13.0,01:40,12:55,9.4,11.6,51.5,13:50,SSE
6,6,1,Winter,11.8,14.4,13.1,19:35,00:05,38.9,9.9,57.9,08:10,SSE


In [36]:
class(weather$season)

[1] "character"

In [37]:
summary(weather$season)

   Length     Class      Mode 
      365 character character 

In [38]:
weather$season <- factor(weather$season,
                         levels = c("Spring","Summer","Autumn","Winter"))

In [39]:
class(weather$season)

[1] "factor"

In [40]:
summary(weather$season)


Spring Summer Autumn Winter 
    92     92     91     90

In [41]:
weather$day <- as.factor(weather$day)
weather$month <- as.factor(weather$month)
weather$dir.wind <- as.factor(weather$dir.wind)

In [42]:
rel <- round(prop.table(table(weather$dir.wind))*100,1)

In [43]:
rel


   E  ENE  ESE    N   NE  NNE  NNW   NW    S   SE  SSE  SSW   SW    W  WNW  WSW 
 3.0  4.1  0.5  4.9  6.8  2.2 10.1 29.6  7.1  6.6  8.5  4.7  3.0  1.4  6.6  0.8 

In [44]:
weather$dir.wind.8 <- weather$dir.wind 
weather$dir.wind.8 <- ifelse(weather$dir.wind %in%  c("NNE","ENE"),"NE",as.character(weather$dir.wind.8)) 
weather$dir.wind.8 <- ifelse(weather$dir.wind %in% c("NNW","WNW"),"NW",as.character(weather$dir.wind.8)) 
weather$dir.wind.8 <- ifelse(weather$dir.wind %in% c("WSW","SSW"),"SW",as.character(weather$dir.wind.8)) 
weather$dir.wind.8 <- ifelse(weather$dir.wind %in% c("ESE","SSE"), "SE",as.character(weather$dir.wind.8)) 
weather$dir.wind.8 <- factor(weather$dir.wind.8,levels = c("N","NE","E","SE","S","SW","W","NW"))

In [45]:
round(prop.table(table(weather$dir.wind.8,weather$season),margin = 2)*100,1)

    
     Spring Summer Autumn Winter
  N     1.1    3.3   12.1    3.3
  NE   14.1    5.4   20.9   12.2
  E     0.0    0.0    5.5    6.7
  SE   13.0   14.1   20.9   14.4
  S     5.4   12.0    4.4    6.7
  SW    6.5    8.7    2.2   16.7
  W     2.2    0.0    1.1    2.2
  NW   57.6   56.5   33.0   37.8

In [46]:
first.day <- as.Date("2014-01-01")
class(first.day)

[1] "Date"

In [47]:
weather$date  <- first.day + weather$day.count - 1 

# B) Prediccion de la cantidad de lluvia diaria usando multiple linear regression
Creating a model from inferential statistics using multiple linear regression

## B.1) Training and test datasets 

In [48]:
set.seed(123) 

In [49]:
# randomly pick 70% of the number of observations (365)
index <- sample(1:nrow(weather),size = 0.7*nrow(weather)) 
 
# subset weather to include only the elements in the index
train <- weather[index,] 
 
# subset weather to include all but the elements in the index
test <- weather [-index,] 

In [50]:
nrow(train)
nrow(test)

[1] 255

[1] 110

## Teoria: Metricas de error

### RMSE 
La métrica de error principal que utilizaremos para evaluar nuestros modelos es el RMSE (error de la raíz cuadrada media). 
Esta medida de error otorga más importancia a los residuos más grandes que a los más pequeños (un residuo es la diferencia entre el valor previsto y el observado). 
Significa es que consideramos que omitir la predicción de la cantidad de lluvia en 20 mm, en un día determinado, no solo es el doble de malo que faltar en 10 mm, sino que es peor que eso.

### MAE
El MAE (error absoluto medio) sera la métrica de error secundaria. Da igual peso a los residuos, lo que significa que 20 mm es en realidad el doble de malo que 10 mm. Una de las ventajas de esta medida de error es que es fácil de interpretar: nos dice, en promedio, la magnitud del error que obtenemos al usar el modelo en comparación con los valores reales observados.

### RMSE & MAE
Ambas métricas son válidas, aunque el RMSE parece ser más popular, posiblemente porque amplía las diferencias entre los rendimientos de los modelos en situaciones en las que el MAE podría llevarnos a creer que eran casi iguales. 

Link: https://www.geosci-model-dev.net/7/1247/2014/gmd-7-1247-2014.pdf

Link II: stackoverflow

## B.2) Set the baseline

What would be our best guess if we had to predict the amount of rain?
El promedio de todas las lluvias.

In [51]:
best.guess <- mean(train$rain) 
best.guess

[1] 5.315294

In [52]:
# Evaluate RMSE and MAE on the testing data
RMSE.baseline <- sqrt(mean((best.guess-test$rain)^2))
RMSE.baseline

[1] 13.38996

In [53]:
MAE.baseline <- mean(abs(best.guess-test$rain))
MAE.baseline

[1] 8.219123

## Teoria: Multiple linear regression

Los modelos lineales no requieren que las variables tengan una distribución gaussiana (solo los errores / residuales deben distribuirse normalmente); Sin embargo, sí requieren una relación lineal entre las variables dependientes e independientes.

Por esta razón de linealidad, y también para ayudar a solucionar el problema con los residuos que tienen una varianza no constante en el rango de predicciones (denominada heterocedasticidad), realizaremos la transformación de registro habitual en la variable dependiente. Como tenemos ceros (días sin lluvia), no podemos hacer una transformación simple de ln (x), pero sí podemos hacer ln (x + 1), donde x es la cantidad de lluvia. 

# #EsPregunta

A) realmente se hacen estas preguntas en general? Tipo voy a verificar que mis variables tengan una distribucion gaussiana? Y de ser asi como lo verifican?

b) "Por esta razón de linealidad, y también para ayudar a solucionar el problema con los residuos que tienen una varianza no constante en el rango de predicciones (denominada heterocedasticidad), realizaremos la transformación de registro habitual en la variable dependiente. Como tenemos ceros (días sin lluvia), no podemos hacer una transformación simple de ln (x), pero sí podemos hacer ln (x + 1), donde x es la cantidad de lluvia"

Pregunta:
Solo puedo decir que tengo dias sin lluvia, por eso dice que tiene una varianza no constante? Porque algunos de sus datos van a cero?

c) ¿Por qué elige aplicar una función logarítmica? Porque usa log-linear?


## Teoria: Como funciona el proceso de eliminacion de variables predictoras?

Comencé con todas las variables como predictores potenciales y luego eliminé del modelo, una por una, aquellas que no eran estadísticamente significativas (p <0,05). Se eliminan de a uno por uno debido a la multicolinealidad (es decir, la correlación entre variables independientes). 

Algunas de las variables en nuestros datos están altamente correlacionadas (por ejemplo, la temperatura mínima, promedio y máxima en un día determinado), lo que significa que a veces cuando eliminamos una variable no significativa del modelo, otra que anteriormente no era -significante se vuelve estadísticamente significativo.

Este proceso iterativo de eliminación hacia atrás se detiene cuando todas las variables en el modelo son significativas (en el caso de factores, aquí consideramos que al menos un nivel debe ser significativo);

## B.3) Create multiple (log)linear regression model using the training data

In [62]:
lin.reg <- lm(log(rain+1) ~ season +  h.temp + ave.temp + ave.wind + gust.wind +
              dir.wind, data = train)

In [55]:
summary(lin.reg)


Call:
lm(formula = log(rain + 1) ~ season + h.temp + ave.temp + ave.wind + 
    gust.wind + dir.wind, data = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.87548 -0.50485 -0.01998  0.41785  2.24267 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.278598   0.430907  -0.647 0.518570    
seasonSummer  0.398979   0.153002   2.608 0.009707 ** 
seasonAutumn  0.732204   0.153985   4.755 3.49e-06 ***
seasonWinter  0.319533   0.160923   1.986 0.048253 *  
h.temp       -0.199797   0.046376  -4.308 2.43e-05 ***
ave.temp      0.164593   0.055452   2.968 0.003309 ** 
ave.wind     -0.119636   0.041364  -2.892 0.004188 ** 
gust.wind     0.065260   0.008599   7.589 7.79e-13 ***
dir.windENE   0.232046   0.343492   0.676 0.499999    
dir.windESE   0.343658   0.804671   0.427 0.669718    
dir.windN     0.320433   0.334591   0.958 0.339218    
dir.windNE    0.374025   0.342513   1.092 0.275965    
dir.windNNE   1.752912   0.512710   3.419 0.00074

In [56]:
exp(lin.reg$coefficients["gust.wind"])

gust.wind 
 1.067436

In [57]:
test.pred.lin <- exp(predict(lin.reg,test))-1
test.pred.lin

2            4            5            6            7           10 
 9.469867638 19.050322506 13.565093442 27.676818064  7.534265142  0.774383777 
          11           12           13           14           18           19 
-0.057603629 12.505307483  3.981839540  5.368455579  1.782855939  2.114125097 
          21           23           25           28           31           33 
 5.209238556  0.676561068  1.104984622 21.243885199 13.143730206 17.473719510 
          42           43           55           57           60           62 
 4.264009261 10.652915149 11.054638827 10.079900072 18.999942224  9.998245209 
          63           66           74           81           89           91 
 5.003656092 -0.386577873 -0.328397070  3.160521119  5.711313927  8.877237243 
          95           97          104          112          113          116 
 2.571543923  0.949084328  0.599261017  3.010655700  1.006171138  1.151044670 
         124          126          133          136          140          142 
-0.299835831 -0.101896607 -0.291442430  2.012721992  8.796858473  1.974746462 
         143          147          150          153          154          155 
 0.799142074  0.135327852  1.039039020  0.079769007  0.359690914  0.571065843 
         165          168          170          173          178          182 
-0.578089880  0.092674614 -0.189656032  2.658670068  0.864964523  1.137128573 
         186          188          191          192          194          195 
 3.065783286  2.298572754  0.030797589  6.664728395  0.130869721 -0.075480217 
         198          199          206          207          215          217 
 1.472217581  4.862355318 -0.221363907  2.255453253  2.741388144  0.662197565 
         218          221          227          229          230          234 
 0.714190260  3.569963849  0.458194045 -0.361577500  2.676131657  0.221869914 
         235          244          247          252          253          254 
 0.145156790 -0.369650030  0.302935033 -0.149071584  0.870957357  5.648016803 
         261          262          264          269          273          278 
15.657794936  8.071390756 24.798725688  0.263314610  0.255937568  1.291055273 
         280          281          283          285          286          289 
 6.267079080 10.473560650  0.262935412  0.930929372 27.989866938 33.027093943 
         290          292          294          296          297          303 
17.053807749 -0.161188398 -0.472470389 -0.100471238 -0.184383861  0.589485508 
         305          318          325          335          336          338 
 1.704554530 28.845504646  0.314865953  2.682228470  1.054944960  2.348547776 
         342          343          346          349          350          352 
 0.870247642  1.679075917 17.115676231  2.143495813  1.939689005  7.549953315 
         355          357 
 0.006572807  0.612155261

In [58]:
RMSE.lin.reg <- sqrt(mean((test.pred.lin-test$rain)^2))
RMSE.lin.reg

[1] 10.24922

In [59]:
MAE.lin.reg <- mean(abs(test.pred.lin-test$rain))
MAE.lin.reg

[1] 4.787961

## Conclusiones 

1) Tanto el RMSE como el MAE han disminuido significativamente en comparación con el modelo de referencia, lo que significa que este modelo lineal, a pesar de todos los problemas de linealidad y el hecho de que predice valores negativos de lluvia en algunos días, sigue siendo mucho mejor, en general, que nuestro mejor conjetura.

2) Todas las variables son estadísticamente significativas (p <0.05), como se esperaba de la forma en que se construyó el modelo, y el predictor más significativo es la ráfaga de viento (p = 7.44e-12). 


## #EsPregunta

### A) 
El predictor mas significativo sera aquel con p menor. Porque #EsPregunta_A

### B)
En este modelo: Multiple R-squared: 0.6554, Adjusted R-squared: 0.6228
Puedo decir lo mismo: que el Multiple R-squared original no se ha aumentado agregando variables al modelo?